In [8]:
import requests as req
from bs4 import BeautifulSoup

In [10]:
url = "https://id.tradingview.com/markets/stocks-indonesia/market-movers-all-stocks/"
page = req.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="is-not-authenticated is-not-pro is-not-trial" dir="ltr" lang="id">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
  <script nonce="QS5t9qiJYq4N7gxc6mPQ9g==">
   window.initData = {};
  </script>
  <link href="https://notifications.tradingview.com/news/channel" rel="news-streaming-url"/>
  <title>
   Semua Saham Indonesia dalam Satu Halaman — TradingView
  </title>
  <link crossorigin="use-credentials" href="https://pricealerts.tradingview.com" rel="preconnect"/>
  <!-- render_css_bundle('big_footer') -->
  <link crossorigin="anonymous" href="https://static.tradingview.com/static/bundles/65857.aa61e41499c884471418.css" rel="stylesheet" type="text/css">
   <link crossorigin="anonymous" href="https://static.tradingview.com/static/bundles/36010.a1a91e4e3b8d4d56af6a.css" rel="stylesheet" type="text/css">
    <link crossorigin="anonymous" href="htt